The following code will show you how to transform the data into relevancy. All categorical features will become numeric, missing data will be either dropped or appropriately replaced.

In [1]:
# Import relevant modules and load your dataframe.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from range_key_dict import RangeKeyDict

df = pd.read_csv('/Users/Haebichan/Desktop/Harvard University Sample.csv', header = None)

df.columns = 'gender type_of_high_school state unweighted_gpa weighted_gpa sat_math sat_reading sat_writing act_total school_ranking decision'.split()

In [2]:
df.head()

,gender,type_of_high_school,state,unweighted_gpa,weighted_gpa,sat_math,sat_reading,sat_writing,act_total,school_ranking,decision
0,Female,Public,CA,3.75,4.30,800.0,740.0,NaN,NaN,10 of 370,Denied
1,Female,Public,CA,4.00,4.33,770.0,800.0,730.0,NaN,7 of 598,Accepted
2,Female,Private,TX,4.00,NaN,750.0,740.0,740.0,33.0,13 of 252,Denied
3,Female,Private,TX,4.00,NaN,660.0,800.0,770.0,NaN,DR,Wait-Listed
4,Female,Public,CA,4.00,5.00,750.0,750.0,800.0,NaN,Top 10%,Accepted


In [3]:
# Manually replace all categorical variables into cat codes.

df.gender = df.gender.map({'Female':0, 'Male':1})
df.type_of_high_school = df.type_of_high_school.map({'Public':0, 'Private':1})
df.replace(['Accepted', 'Will Attend','Denied'], [1,1,0], inplace=True)

In [4]:
# Drop rows with wait-listed samples as we can't use them; there just aren't enough of them to contribute to classification prediction. 
# Drop rows with a missing state value.
# Drop the unweighted_gpa column, as many are missing. Use (with caution) the .interpolate() function or a machine learning technique to fill in the missing values, if you wish. 

df = df[df.decision != 'Wait-Listed']

df = df[df.state.notnull()]

df.drop('weighted_gpa',1,inplace=True)

In [5]:
# Drop rows without SAT scores in any of the three categories. Create a column with total SAT scores. 
# Drop total SAT scores that are too low and are many standard deviations away from the mean. 

df = df[(df['sat_writing'].notnull()) & (df['sat_math'].notnull()) & (df['sat_reading'].notnull())]

df['sat_total'] = df.sat_math + df.sat_reading + df.sat_writing
df = df[(df.sat_total >= 1590) | (df.act_total >= 22)]

In [6]:
# Make sure a row contains either sat_total value or the act_total value. If both values are missing, the row becomes useless.

df = df[(df.sat_total.notnull()) | (df.act_total.notnull())]


In [7]:
# Multiply the GPA to make them into integers to summon the RangeKeyDict function (Range can't take in floats). I will return the numbers back to normal later.
# Another possibility is to create a separate function where the range can take in floats. This will be a little more work.

df.unweighted_gpa = df.unweighted_gpa * 100

In [8]:
# return only rows where unweighted_gpa is not a NaN. GPA is a crucial part of the classification prediction to be left blank.


df = df[df.unweighted_gpa.notnull()]

df.replace(['DR'], np.NaN, inplace=True)

In [9]:
# Create a dictionary for rankings and its corresponding gpa using the RangeKeyDict.
### I calculated the range manually. I took the school rankings without the 'DR', recorded all the GPA and its corresponding school ranking, and used its range as the guide for this smaller set. I then applied the ranking_dict to the missing values.
# Finally, return the GPA to its normal float number by division. 

ranking_dict = RangeKeyDict({
        (400, 401): 'Top 1%', 
        (391, 400): 'Top 5%', 
        (381, 391): 'Top 10%', 
        (364, 381): 'Top 25%', 
        (250, 364): 'Top 50%',
    })


df.loc[df['school_ranking'].isnull(), 'school_ranking'] = df.loc[df['school_ranking'].isnull(), 'unweighted_gpa'].apply(lambda x: ranking_dict[x])

df.unweighted_gpa = df.unweighted_gpa / 100 

In [10]:
# Not all rankings are stated as "Top X%" - in fact, these rankings are a minority. Most are written as "X out of Y".
# Transform these "X out of Y" rankings as "Top X%" by iterating through the column and transforming strings into percentages.
# Then, map these percentages into the "Top X%" string format. 
# Finally, manually transform these "Top X%" strings into numeric cat codes.

for index, row in df['school_ranking'].items():
    row = str(row)
    if 'of' not in row:
        pass
    else:
        target_word = ['of']
        numbers = row.split()
        result_word = [word for word in numbers if word not in target_word]
        division = int(result_word[0]) / int(result_word[1])
        division = float(division)
        
        if 0.25 < division < 0.50:
            division = 'Top 50%'
        elif 0.10 < division < 0.25:
            division = 'Top 25%'
        elif 0.05 < division < 0.10:
            division = 'Top 10%'
        elif 0.015 < division <= 0.05:
            division = 'Top 5%'
        elif division <= 0.015:
            division = 'Top 1%'
        else:
            pass
        df['school_ranking'][index] = division

df = df.replace(['Top 1%','Top 5%', 'Top 10%', 'Top 25%', 'Top 50%'], [0,1,2,3,4])

/Users/Haebichan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
# Create two dictionaries for rows with missing SAT and ACT scores. The first dictionary will be mapped onto rows without SAT scores. The missing scores will be given by converting the ACT scores given the first dictionary. The opposite effect will take place to the converse situation using the second dictionary. 

score_dict = {36: 2390, 35:2330, 34: 2260, 33: 2170, 32: 2110, 31: 2040, 30: 1990, 29: 1920, 28: 1850, 27: 1810, 26: 1760, 25: 1700, 24: 1650, 23: 1590}

score_dict2 = RangeKeyDict({
    (2380,2401): 36, (2290,2371): 35, (2220,2281): 34, (2140,2211): 33, 
    (2080,2131): 32, (2020,2071): 31, (1980,2011): 30, (1920,1971): 29,
    (1860,1911): 28, (1800,1851): 27, (1740,1791): 26, (1680,1731): 25, 
    (1620,1671): 24, (1560,1611): 23, 
})

df.loc[df['act_total'].isnull(), 'act_total'] = df.loc[df['act_total'].isnull(),'sat_total'].apply(lambda x: score_dict2[x])
df.loc[df['sat_total'].isnull(), 'sat_total'] = df.loc[df['sat_total'].isnull(),'act_total'].apply(lambda x: score_dict[x])


In [12]:
# Take in an input from the user on the acronym of the state of the university in question. Since the example here is Harvard University, the user will type 'MA' (Massachusetts), which will be replaced with the word 'In-State'. All other acronyms will be made turned into 'Out-of-State' string.
        
name_of_state = input('Input the State of the University in question in its acronym \n > ')

for index, row in df.state.items():
    if row == name_of_state:
        row = row.replace(name_of_state, 'In-State')
    elif row != name_of_state:
        row = row.replace(row, 'Out-of-State')
    else:
        pass
    df.state[index] = row

Input the State of the University in question in its acronym 
 > MA


/Users/Haebichan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [13]:
# Manually cat code the strings.

df.state.replace({'In-State': 0, 'Out-of-State': 1}, inplace=True)

In [14]:
df.reset_index(inplace=True)
df.drop('index',1,inplace=True)

In [15]:
df.head()

,gender,type_of_high_school,state,unweighted_gpa,sat_math,sat_reading,sat_writing,act_total,school_ranking,decision,sat_total
0,0,0,1,4.00,770.0,800.0,730.0,35.0,0,1,2300.0
1,0,1,1,4.00,750.0,740.0,740.0,33.0,2,0,2230.0
2,0,0,1,4.00,750.0,750.0,800.0,35.0,2,1,2300.0
3,0,0,1,4.00,700.0,710.0,750.0,34.0,1,0,2160.0
4,0,0,1,3.74,620.0,640.0,670.0,29.0,2,0,1930.0


In [ ]:
df.to_csv('/dataset/Processed Harvard Data.csv')